<a href="https://colab.research.google.com/github/claredavies/AutomaticLearning/blob/main/MachineLearningDataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [111]:
import six
import sys
sys.modules['sklearn.externals.six'] = six

import joblib
sys.modules['sklearn.externals.joblib'] = joblib

from scipy.io import arff
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from sklearn.decomposition import PCA

RANDOM_SEED = 0

# Functions

In [2]:
def read_arff_as_df_scipy( filename ):
 # input : name of the file where the dataset is stored
 # output : a dataframe
 dataset = arff.loadarff(filename)
 df = pd.DataFrame( dataset[0] , columns = dataset[1])
 return df

In [3]:
def return_columns_missing_values(df):
  return df.columns[df.isnull().any()]

In [15]:
def missing_value_column_percentage(dataframe, column_name):
  percent_missing = dataframe.loc[:, column_name].isnull().sum() * 100 / len(dataframe)
  return percent_missing

In [28]:
def convert_object_datatypes_to_categorical(dataframe):
  for column_name in dataframe:
    if(dataframe[column_name].dtype == object):
      dataframe[column_name] = dataframe[column_name].astype('category')
  return dataframe

In [35]:
def convert_categorical_to_integer(dataframe):
  cat_columns = dataframe.select_dtypes(['category']).columns
  dataframe[cat_columns] = dataframe[cat_columns].apply(lambda x: x.cat.codes)
  return dataframe

# Read in Data

In [127]:
df = read_arff_as_df_scipy('dataset_55_hepatitis.arff')

# Data Exploration

In [128]:
print(df.info)

<bound method DataFrame.info of       AGE        SEX STEROID ANTIVIRALS FATIGUE MALAISE ANOREXIA LIVER_BIG  \
0    30.0    b'male'   b'no'      b'no'   b'no'   b'no'    b'no'     b'no'   
1    50.0  b'female'   b'no'      b'no'  b'yes'   b'no'    b'no'     b'no'   
2    78.0  b'female'  b'yes'      b'no'  b'yes'   b'no'    b'no'    b'yes'   
3    31.0  b'female'    b'?'     b'yes'   b'no'   b'no'    b'no'    b'yes'   
4    34.0  b'female'  b'yes'      b'no'   b'no'   b'no'    b'no'    b'yes'   
..    ...        ...     ...        ...     ...     ...      ...       ...   
150  46.0  b'female'  b'yes'      b'no'  b'yes'  b'yes'   b'yes'    b'yes'   
151  44.0  b'female'  b'yes'      b'no'  b'yes'   b'no'    b'no'    b'yes'   
152  61.0  b'female'   b'no'      b'no'  b'yes'  b'yes'    b'no'     b'no'   
153  53.0    b'male'   b'no'      b'no'  b'yes'   b'no'    b'no'    b'yes'   
154  43.0  b'female'  b'yes'      b'no'  b'yes'   b'no'    b'no'    b'yes'   

    LIVER_FIRM SPLEEN_PALPABLE 

In [129]:
print(df.head)

<bound method NDFrame.head of       AGE        SEX STEROID ANTIVIRALS FATIGUE MALAISE ANOREXIA LIVER_BIG  \
0    30.0    b'male'   b'no'      b'no'   b'no'   b'no'    b'no'     b'no'   
1    50.0  b'female'   b'no'      b'no'  b'yes'   b'no'    b'no'     b'no'   
2    78.0  b'female'  b'yes'      b'no'  b'yes'   b'no'    b'no'    b'yes'   
3    31.0  b'female'    b'?'     b'yes'   b'no'   b'no'    b'no'    b'yes'   
4    34.0  b'female'  b'yes'      b'no'   b'no'   b'no'    b'no'    b'yes'   
..    ...        ...     ...        ...     ...     ...      ...       ...   
150  46.0  b'female'  b'yes'      b'no'  b'yes'  b'yes'   b'yes'    b'yes'   
151  44.0  b'female'  b'yes'      b'no'  b'yes'   b'no'    b'no'    b'yes'   
152  61.0  b'female'   b'no'      b'no'  b'yes'  b'yes'    b'no'     b'no'   
153  53.0    b'male'   b'no'      b'no'  b'yes'   b'no'    b'no'    b'yes'   
154  43.0  b'female'  b'yes'      b'no'  b'yes'   b'no'    b'no'    b'yes'   

    LIVER_FIRM SPLEEN_PALPABLE SP

Get number of samples and features and store

In [130]:
no_samples = len(df.index)
no_features = len(df.columns)
print(no_samples)
print(no_features)

155
20


Check which columns have missing values

In [131]:
columns_missing_values = return_columns_missing_values(df)
print(columns_missing_values)

Index(['BILIRUBIN', 'ALK_PHOSPHATE', 'SGOT', 'ALBUMIN', 'PROTIME'], dtype='object')


# Data Cleaning

Drop features with more than 40% of missing values

In [132]:
for column_name in columns_missing_values:
  column_missing_values_percentage = missing_value_column_percentage(df, column_name)
  if(column_missing_values_percentage > 40):
    df = df.drop(column_name, axis=1)

In [133]:
# drop missing values
df1 = df.dropna()
# fill missing values with median
df2 = df.fillna(df.median())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.


 Transform categorical features into integer-valued features, using progressive integer labels. You have to do it
for both df1 and df2.

In [134]:
df1 = convert_object_datatypes_to_categorical(df1)
df2 = convert_object_datatypes_to_categorical(df2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [135]:
df1.dtypes

AGE                 float64
SEX                category
STEROID            category
ANTIVIRALS         category
FATIGUE            category
MALAISE            category
ANOREXIA           category
LIVER_BIG          category
LIVER_FIRM         category
SPLEEN_PALPABLE    category
SPIDERS            category
ASCITES            category
VARICES            category
BILIRUBIN           float64
ALK_PHOSPHATE       float64
SGOT                float64
ALBUMIN             float64
HISTOLOGY          category
Class              category
dtype: object

In [136]:
df1 = convert_categorical_to_integer(df1)
df2 = convert_categorical_to_integer(df2)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]



# Normalisation

In [137]:
y1 = df1['Class']
X1 = df1.drop("Class", axis=1)

y2 = df2['Class']
X2 = df2.drop("Class", axis=1)

standardizer = StandardScaler()
X1 = standardizer.fit_transform(X1)
X2 = standardizer.fit_transform(X2)

In [138]:
print(y1.head(100))

0      1
1      1
2      1
3      1
5      1
      ..
124    1
125    1
127    0
128    1
129    0
Name: Class, Length: 100, dtype: int8



# Feature Selection

In [139]:
knn = KNeighborsClassifier(n_neighbors=4) # ml_algo used = knn

fss1 = SFS(knn,
k_features=5,
forward=True, # if forward = True then SFS otherwise SBS
floating=False,
verbose=2,
scoring='accuracy'
)

bss1 = SFS(knn,
k_features=5,
forward=False, # if forward = True then SFS otherwise SBS
floating=False,
verbose=2,
scoring='accuracy'
)

fss2 = SFS(knn,
k_features=5,
forward=True, # if forward = True then SFS otherwise SBS
floating=False,
verbose=2,
scoring='accuracy'
)

bss2 = SFS(knn,
k_features=5,
forward=False, # if forward = True then SFS otherwise SBS
floating=False,
verbose=2,
scoring='accuracy'
)

#after applying sfs fit the data:
fss1.fit(X1, y1)
bss1.fit(X1,y1)

fss2.fit(X1, y1)
bss2.fit(X1,y1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.2s finished

[2022-10-17 15:27:52] Features: 1/5 -- score: 0.8666666666666668[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.2s finished

[2022-10-17 15:27:52] Features: 2/5 -- score: 0.8666666666666668[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.2s finished

[2022-10-17 15:27:53] Features: 3/5 -- score: 0.8666666666666668[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

SequentialFeatureSelector(estimator=KNeighborsClassifier(n_neighbors=4),
                          forward=False, k_features=5, scoring='accuracy',
                          verbose=2)

# PCA (feature reduction)

In [140]:
pca = PCA(n_components=5)
principalComponents1 = pca.fit_transform(X1)

principal_component_df1 = pd.DataFrame(data = principalComponents1
             , columns = ['principal component 1', 'principal component 2','principal component 3', 'principal component 4','principal component 5'])

principal_component_df1.tail()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5
115,-1.220112,-1.212932,-1.094972,0.661552,-0.313786
116,-0.080074,-0.205704,-0.456207,1.050548,0.357005
117,1.067231,0.857912,-0.079264,-1.547214,0.850535
118,1.557447,-1.777749,0.676933,-0.271041,1.449051
119,1.980189,-2.156425,-1.077784,-0.665747,-0.290179


principal component 1 holds 21.8% of the information while the principal component 2 holds only 12.7% of the information

In [141]:
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

Explained variation per principal component: [0.21837298 0.12725674 0.08343554 0.0714942  0.06564351]


# Compare Performance

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=RANDOM_SEED)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=RANDOM_SEED)